In [ ]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from tqdm import tqdm

users = pd.read_csv('../../../dataset/userid-profile.tsv', sep='\t', header=0, index_col='#id')

listening_history = pd.read_csv('../../../dataset/userid-timestamp-artid-artname-traid-traname.tsv', sep='\t', encoding='utf8', header=None, error_bad_lines=False)
listening_history.columns = ["#id", "timestamp ", "artist-id", "artist-name", "track-id","track-name"]

In [ ]:
# get_user_id
users_us = users[users['country']=='United States'].reset_index()

In [ ]:
# reset index
listening_history_us = listening_history.set_index('#id').loc[users_us['#id']]
listening_history_us = listening_history_us.reset_index()

In [ ]:
#calculate  the distirbution of the users
unique_users = listening_history_us['#id'].unique()
unique_tracks = listening_history_us['track-name'].unique()
print ('Number of unique users : ' + str(len(unique_users)))
print ('Number of unique tracks : ' + str(len(unique_tracks)))
print ('Times of tracks being listened: ' + str(len(listening_history_us)))

In [ ]:
# remap the users
track_keys = {}

for track in unique_tracks:
    track_keys[track] = len(track_keys)
    
user_keys = {}

for user in unique_users:
    user_keys[user] = len(user_keys)
    
unique_user = len(unique_users)
unique_track = len(unique_tracks)
    

In [ ]:
listening_history_us['tid']= listening_history_us['track-name'].map(track_keys)
listening_history_us['uid']= listening_history_us['#id'].map(user_keys)

In [ ]:
lh = listening_history_us[['uid','tid']]

In [ ]:
lh['count'] = 1

In [ ]:
lh = lh.groupby(["uid", "tid"]).sum().reset_index()
lh[:10]

In [ ]:
lh.to_pickle('../data/counting.pkl')

In [ ]:
max_list = []
for i in lh['uid'].unique():
    user = lh[lh['uid'] == i]
    max_value = max(user['count'])
    max_list.append(max_value)

In [ ]:
counting_list = []
for i in range(0, unique_user):
    counting_list.append(np.zeros(unique_track, dtype=np.int))

In [ ]:
for i in tqdm(range(unique_track)):
    temp = lh[lh['tid'] == i]
    for index, row in temp.iterrows():
        uindex = int(row['uid'])
        counting_list[uindex][i] = row['count']

In [ ]:
np.savetxt("../data/counting_list.csv", counting_list, delimiter=",", fmt='%d')

In [ ]:
#minmax

In [ ]:
normalized_list = []
for i in range(0, unique_user):
    normalized_list.append(np.zeros(unique_track, dtype=np.int))

In [ ]:
for i in tqdm(lh['uid'].unique()):
    user = lh[lh['uid'] == i]
    max_value = max(user['count'])
    normalized_list[i] = [j *5 / max_value for j in counting_list[i]]

In [ ]:
np.savetxt("../data/normalized_list.csv", normalized_list, delimiter=",")

In [ ]:
counting_list

In [ ]:
normalized_list

In [ ]:
normalized_pd = lh.copy()
normalized_pd.rename(columns={'count':'rating'}, inplace=True)
normalized_pd = normalized_pd.astype({'rating': 'float64'})

In [ ]:
for i in tqdm(range(unique_user)):
    temp = lh[lh['uid'] == i]
    for index, row in temp.iterrows():
        normalized_pd['rating'][index] /= max_list[i]
        normalized_pd['rating'][index] *= 5

In [ ]:
normalized_pd.to_pickle('../data/normalized_pd.pkl')